O código é um _WordCounter_ desenvolvido para entrega de desafio da plataforma Digital Innovation One e será utilizado aqui para aplicação de libs para a tornar mais performática.
Código original:

In [ ]:
import re, sys

def main():
	parametros = sys.argv
	msg_err = f'Necessário o uso dos parâmetros. Tente: "python3 {parametros[0]} <path_leitura> <path_saída>"'

	try:
		if len(parametros) != 3: raise FileNotFoundError
		
		arquivo = open(parametros[1], 'r', encoding='utf8')
		dados = arquivo.read()
		arquivo.close()

		dados = re.findall("[\w]+", dados)
		dados = [i.lower() for i in dados]
		dados = [(i, dados.count(i)) for i in set(dados)]
		dados.sort(key=lambda x: x[1], reverse=True)

		arquivo_saida = open(parametros[2], 'a', encoding='utf8')

		for i in dados:
			arquivo_saida.write(f'{i[0]}: {i[1]}\n')
				
		arquivo_saida.close()

	except FileNotFoundError:
		print(msg_err)

if __name__ == '__main__':
	#main()
	pass

Com o único propósito de estressar o processamento, será adicionada a lista 'dados_brutos', o que possibilitará a implantação de paralelismo

In [ ]:
import re, sys

def main():
	parametros = sys.argv
	msg_err = f'Necessário o uso dos parâmetros. Tente: "python3 {parametros[0]} <path_leitura> <path_saída>"'

	try:
		if len(parametros) != 3: raise FileNotFoundError
		
		arquivo = open(parametros[1], 'r', encoding='utf8')
		dados = dados_brutos = arquivo.read()
		arquivo.close()

		dados = re.findall("[\w]+", dados)
		dados = [i.lower() for i in dados]
		dados = [(i, dados.count(i)) for i in set(dados)]
		dados.sort(key=lambda x: x[1], reverse=True)

		dados_brutos = dados_brutos.split()                                         #
		dados_brutos = [(i, dados_brutos.count(i)) for i in set(dados_brutos)]      #
		dados_brutos.sort(key=lambda x: x[1], reverse=True)                         #

		arquivo_saida = open(parametros[2], 'a', encoding='utf8')
		arquivo_saida.write('Re'.ljust(30)+ '| Brutos\n'+'-'*100 + '\n')

		for i in range(len(dados_brutos)):
			try:	# Essa estrutura, apesar de não cumprir o propósito original, é um pouco mais performática do que if/else 
				arquivo_saida.write(f'{dados[i][0]}: {dados[i][1]}'.ljust(30) + '| ' + f'{dados_brutos[i][0]}: {dados_brutos[i][1]}\n')
			except IndexError:
				arquivo_saida.write(' '*30 + '| ' + f'{dados_brutos[i][0]}: {dados_brutos[i][1]}\n')
				
		arquivo_saida.close()

	except FileNotFoundError:
		print(msg_err)

if __name__ == '__main__':
	#main() 
    pass

Esse código foi originalmente concebido para aceitar parâmetros do CLI e, para rodar no Jupiter, tais parâmetros serão inseridos _hardcoded_. Além disso será importado a lib 'time' que possibilitará a avaliação de desempenho do programa.

In [ ]:
import re, time #sys não é mais necessário

def main():
    start = time.time()     #

    parametros = ('filename', 'sherlock.txt', 'saida.txt') # o que dispensa a estrutura try/except
    
    arquivo = open(parametros[1], 'r', encoding='utf8')
    dados = dados_brutos = arquivo.read()
    arquivo.close()

    dados = re.findall("[\w]+", dados)
    dados = [i.lower() for i in dados]
    dados = [(i, dados.count(i)) for i in set(dados)]
    dados.sort(key=lambda x: x[1], reverse=True)

    dados_brutos = dados_brutos.split()
    dados_brutos = [(i, dados_brutos.count(i)) for i in set(dados_brutos)]
    dados_brutos.sort(key=lambda x: x[1], reverse=True)

    arquivo_saida = open(parametros[2], 'a', encoding='utf8')
    arquivo_saida.write('Re'.ljust(30)+ '| Brutos\n'+'-'*100 + '\n')

    for i in range(len(dados_brutos)):
        try:
            arquivo_saida.write(f'{dados[i][0]}: {dados[i][1]}'.ljust(30) + '| ' + f'{dados_brutos[i][0]}: {dados_brutos[i][1]}\n')
        except IndexError:
            arquivo_saida.write(' '*30 + '| ' + f'{dados_brutos[i][0]}: {dados_brutos[i][1]}\n')
            
    arquivo_saida.close()

    print(time.time() - start)      #

if __name__ == '__main__':
	main()
    

O programa é finalizado em 46s.

Ocorre que o programa é executado em um único _core_ da máquina de forma linear e podemos perceber que o tratamento das variáveis 'dados' e 'dados_brutos' podem ser feitos em paralelo. As opções de paralelismo são _Threading_ e _Multiprocessing_, cujas libs Python recebem mesmo nome. Neste caso dividir a execução desse programa em novas _threads_ não traria benefício performático algum (ou qualquer outro benefício), uma vez que o processamento do código é contínuo, o _core_ fica em 100% de uso durante toda execução. Antes da implantação de _multiprocessing_ alguns conceitos iniciais:

In [ ]:
import multiprocessing as mp

def segundo_processo(parametro: int) -> int:    # A função alvo não pode estar aninhada, deve estar declarada no escopo global
    print(parametro + 100)                      # porque será importada pelo segundo processo

if __name__ == '__main__':  # essa estrutura é 'obrigatória' pois o programa será importado
    integral = 10

    mp.Process(target=segundo_processo, args=(integral,)).start() # args recebe uma tupla cujo default é ()

    print(integral)

A execução no jupiter nb trás apenas o 'print' do processo principal, a execução no terminal ou no prompt trará:
~~~Python
10
110
~~~
Deve-se observar o atraso ao iniciar um novo processo, o que justifica o '10' ter sido "printado" antes do '100':

In [ ]:
import multiprocessing as mp
import time

def processo_filho(começo):
    print(time.time() - começo)

if __name__ == '__main__':
    for i in range(5):
        começo = time.time()
        mp.Process(target=processo_filho, args=(começo,)).start()

Output:
~~~Python
0.6875026226043701
0.6250038146972656
0.6406240463256836
0.6718716621398926
0.6718752384185791
~~~
Agora com uma pequena diferença:

In [ ]:
import multiprocessing as mp
import time

def processo_filho(começo):
    print(time.time() - começo)

if __name__ == '__main__':
    for i in range(5):
        começo = time.time()
        p = mp.Process(target=processo_filho, args=(começo,))   #
        p.start()                                               #
        p.join()                                                # 'segura' a execução até que o processo seja concluído

Output:
~~~Python
0.4531240463256836
0.39063072204589844
0.39067554473876953
0.39068102836608887
0.3906219005584717
~~~
Ou seja, a chamada 'simultânea' de vários processos atrasa o retorno deles, mas tem tempo total de execução menor.

Como sabemos, processos diferentes tem memória alocada diferente, ao contrario das threads, que compartilham memória (o que pode causar problemas). Uma das soluções é o uso de _Queues_: 

In [ ]:
import multiprocessing as mp

def p(q, s: str) -> str:
    for i in ['a', 'b', 'c']:	
    	q.put(s+i)

if __name__ == '__main__':
	q = mp.Queue()
	mp.Process(target=p, args=(q,'a')).start()

	print(q.get())
	print('1')
	print(q.get())
	print('2')
	print(q.get())

Output:
~~~Python
aa
1
ab
2
ac
~~~
O uso de Queue é _thread-safe_ e _process-safe_. A 'stack' montada pela Queue é , por default, do tipo FIFO (_First In, First Out_).

Por fim, será implementado o _Multiprocessing_ no código contador de palavras:

In [ ]:
import re, time
import multiprocessing as mp

def processo_extra(q, dados_brutos):
	dados_brutos = dados_brutos.split()
	dados_brutos = [(i, dados_brutos.count(i)) for i in set(dados_brutos)]
	dados_brutos.sort(key=lambda x: x[1], reverse=True)		
	q.put(dados_brutos)

def main():
	start = time.time()

	parametros = ('filename', 'sherlock.txt', 'saida.txt')

	arquivo = open(parametros[1], 'r', encoding='utf8')
	dados = dados_brutos = arquivo.read()
	arquivo.close()

	q = mp.Queue()
	mp.Process(target=processo_extra, args=(q, dados_brutos)).start()

	dados = re.findall("[\w]+", dados)
	dados = [i.lower() for i in dados]
	dados = [(i, dados.count(i)) for i in set(dados)]
	dados.sort(key=lambda x: x[1], reverse=True)

	arquivo_saida = open(parametros[2], 'a', encoding='utf8')
	arquivo_saida.write('Re'.ljust(30)+ '| Brutos\n'+'-'*100 + '\n')

	dados_brutos = q.get()

	for i in range(len(dados_brutos)):
		try:
			arquivo_saida.write(f'{dados[i][0]}: {dados[i][1]}'.ljust(30) + '| ' + f'{dados_brutos[i][0]}: {dados_brutos[i][1]}\n')
		except IndexError:
			arquivo_saida.write(' '*30 + '| ' + f'{dados_brutos[i][0]}: {dados_brutos[i][1]}\n')
			
	arquivo_saida.close()

	print(time.time() - start)

if __name__ == '__main__':
	main()

Output:
´´´33.68143177032471´´´

Cabe reforçar que a execução no jupiter notebook não permite total visualização das funcionalidade pretendidas, sendo recomendada a execução no prompt ou terminal.

O tempo pode variar um pouco, mas já demonstra que essa simples implementação pode tornar o programa mais performático. Além destes exemplos, a lib multiprocessing oferece diversos outros recursos.